# Tutorial 2: **Notebook** - *FVF*

**By 
gLayout Team**

__Content creators:__ Subham Pal, Saptarshi Ghosh

__Content reviewers:__ Mehedi Saligne

___
# Tutorial Objectives

This notebook is a tutorial on-

- **Importing** and **Placement** of FETs and other macros/Pcells with relative coordinates + Placing (and connecting) Via_stack on the ports of the FETs + encircling them with padrings.
- **Routing** between the placed *Vias* with *C_*, *L_*, *Straight_* and *smart* Routes and explaining the differences between these strategies. Particularly explaning the East/West/North/South angles, (For Example, why C has to parallel but L has to be perpendicular sides) and *Placing and connecting PINs for future LVS runs

## **Target** **Block** : **Flipped Voltage Follwer Cell**

The Flipped Voltage Follower (FVF) is a widely used analog building block known for its low output impedance, fast transient response, and ability to deliver large output current while maintaining voltage buffering properties. It is commonly employed in analog circuit design, particularly in output stages, gain boosting, common-mode feedback circuits, and super class-AB OTAs. 

(a) Conventional Voltage follower (common Drain); (b) Flipped voltage follower (FVF).

![](_images/FVF.png)

```bibtex
Domala, N., Sasikala, G. Low power flipped voltage follower current mirror with improved input output impedances. Sādhanā 46, 142 (2021). https://doi.org/10.1007/s12046-021-01665-6
```

## **NetList generation and LVS**
let's go through the step by step procedure to generate LVS and DRC clean layout of a FVF cell.

In [3]:
import os
os.environ["PATH"] = os.environ["TOOLS"]+'/bin:'+ os.environ["PATH"]

In [251]:
from glayout import MappedPDK, sky130 , gf180
#from gdsfactory.cell import cell
from gdsfactory import Component
from gdsfactory.components import text_freetype, rectangle

In [252]:
from glayout import nmos, pmos
from glayout import via_stack
from glayout import rename_ports_by_orientation
from glayout import tapring

In [253]:
from glayout.util.comp_utils import evaluate_bbox, prec_center, prec_ref_center, align_comp_to_port
from glayout.util.port_utils import add_ports_perimeter,print_ports
from glayout.util.snap_to_grid import component_snap_to_grid
from glayout.spice.netlist import Netlist

In [254]:
from glayout.routing.straight_route import straight_route
from glayout.routing.c_route import c_route
from glayout.routing.L_route import L_route

FVF has two fets as shown in the schematic. We call M1 as input fet and M2 as feedback fet. Lets define arguments for the FETs

### 2. Basic Usage of the GLayout Framework
Each generator is a Python function that takes a `MappedPDK` object as a parameter and generates a DRC clean layout for the given PDK. The generator may also accept a set of optional layout parameters such as the width or length of a MOSFET. All parameters are normal Python function arguments.

The generator returns a `GDSFactory.Component` object that can be written to a `.gds` file and viewed using a tool such as Klayout. In this example, the `gdstk` library is used to convert the `.gds` file to an SVG image for viewing.

The pre-PEX SPICE netlist for the component can be viewed using `component.info['netlist'].generate_netlist()`.

In the following example the FET generator `glayout.primitives.fet` is imported and run with both the [Skywater 130](https://skywater-pdk.readthedocs.io/en/main/) and [GF180](https://gf180mcu-pdk.readthedocs.io/en/latest/) PDKs.

#### Demonstration of Basic Layout / Netlist Generation in SKY130 & GF180

In [ ]:
from glayout.flow.primitives.fet import nmos
from glayout.flow.pdk.sky130_mapped import sky130_mapped_pdk as sky130
from glayout.flow.pdk.gf180_mapped import gf180_mapped_pdk as gf180
import gdstk
import svgutils.transform as sg
import IPython.display
from IPython.display import clear_output
import ipywidgets as widgets

# Used to display the results in a grid (notebook only)
left = widgets.Output()
leftSPICE = widgets.Output()
right = widgets.Output()
rightSPICE = widgets.Output()
hide = widgets.Output()

grid = widgets.GridspecLayout(1, 4)
grid[0, 0] = left
grid[0, 1] = leftSPICE
grid[0, 2] = right
grid[0, 3] = rightSPICE
display(grid)

def display_gds(gds_file, scale = 3):
  # Generate an SVG image
  top_level_cell = gdstk.read_gds(gds_file).top_level()[0]
  top_level_cell.write_svg('out.svg')

  # Scale the image for displaying
  fig = sg.fromfile('out.svg')
  fig.set_size((str(float(fig.width) * scale), str(float(fig.height) * scale)))
  fig.save('out.svg')

  # Display the image
  IPython.display.display(IPython.display.SVG('out.svg'))

def display_component(component, scale = 3):
  # Save to a GDS file
  with hide:
    component.write_gds("out.gds")

  display_gds('out.gds', scale)

with hide:
  # Generate the sky130 component
  component_sky130 = nmos(pdk = sky130, fingers=5)
  # Generate the gf180 component
  component_gf180 = nmos(pdk = gf180, fingers=5)

# Display the components' GDS and SPICE netlists
with left:
  print('Skywater 130nm N-MOSFET (fingers = 5)')
  display_component(component_sky130, scale=2.5)
with leftSPICE:
  print('Skywater 130nm SPICE Netlist')
  print(component_sky130.info['netlist'].generate_netlist())

with right:
  print('GF 180nm N-MOSFET (fingers = 5)')
  display_component(component_gf180, scale=2)
with rightSPICE:
  print('GF 180nm SPICE Netlist')
  print(component_gf180.info['netlist'].generate_netlist())

#### Interactive Primitive Generation in SKY130
The following cell demonstrates the different PCell and Utility generators on the Sky130 PDK.

In [ ]:
from glayout.flow.primitives import fet, mimcap, guardring
from glayout.flow.blocks import diff_pair
import ipywidgets as widgets

selection_button = widgets.RadioButtons(
  options=['NMOS', 'PMOS', 'MIM Capacitor', 'Differential Pair', 'Guardring'],
  orientation='horizontal',
  description='Generator:',
  layout=widgets.Layout(position='right')
)
generate_button = widgets.Button(description='Generate', disabled=False)
output = widgets.Output(layout = widgets.Layout(position='left', overflow='visible'))
hide = widgets.Output()

grid = widgets.GridspecLayout(1, 2)
grid[0, 0] = widgets.VBox([selection_button, generate_button])
grid[0, 1] = output

display(grid)

with hide:
  component = fet.nmos(pdk = sky130)
with output:
  print('NMOS')
  display_component(component)

def generate_component(_):
  selected_comp = selection_button.value

  with output:
    clear_output()
    print(f"Generating {selected_comp}...")
  with hide:
    match selected_comp:
      case 'NMOS':
        component = fet.nmos(pdk = sky130)
      case 'PMOS':
        component = fet.pmos(pdk = sky130)
      case 'MIM Capacitor':
        component = mimcap.mimcap(pdk = sky130)
      case 'Differential Pair':
        component = diff_pair.diff_pair(pdk = sky130)
      case 'Guardring':
        component = guardring.tapring(pdk = sky130)
  with output:
    clear_output()
    print(selected_comp)
    display_component(component, 3)

generate_button.on_click(generate_component)

### 3. Tweak the Parameters
These are some of the parameters the NMOS FET generator accepts:
* `width`: The gate width of the FET.
* `length`: The gate length of the FET.
* `fingers`: The number of fingers. Each finger shares the same source/drain.
* `multipliers`: Number of multipliers (a multiplier is a row of fingers).

Run the below cell and use the sliders to adjust the parameters.

In [ ]:
# Default Values
width=3
length=0.2
fingers=4
multipliers=1

# Create sliders
width_slider = widgets.FloatSlider(description = 'Width:', min = 1, max = 5, step = 0.5, value = width)
length_slider = widgets.FloatSlider(description = 'Length:', min = 0.2, max = 1, step = 0.1, value = length)
fingers_slider = widgets.IntSlider(description = 'Fingers:', min = 1, max = 10, value = fingers)
multipliers_slider = widgets.IntSlider(description = 'Multipliers:', min = 1, max = 5, value = multipliers)
generate_button = widgets.Button(description='Generate', disabled=False)

inputs_box = widgets.VBox([width_slider, length_slider, fingers_slider, multipliers_slider, generate_button])

output = widgets.Output(layout = widgets.Layout(position='left', overflow='visible'))
hide = widgets.Output()

grid = widgets.GridspecLayout(1, 2)
grid[0, 0] = inputs_box
grid[0, 1] = output

display(grid)

def generate_component(_):
  width = width_slider.value
  length = length_slider.value
  fingers = fingers_slider.value
  multipliers = multipliers_slider.value

  with output:
    clear_output()
    print(f"Generating with width={width}, length={length}, fingers={fingers}, multipliers={multipliers}...")
  with hide:
    component = component = fet.nmos(pdk = sky130, width = width, length=length, fingers = fingers, multipliers = multipliers)
  with output:
    clear_output()
    print(f"N-MOSFET with width={width}, length={length}, fingers={fingers}, multipliers={multipliers}:")
    display_component(component)

generate_component(None)

# Regenerate upon change in value
generate_button.on_click(generate_component)

In [ ]:
def fvf_netlist(fet_1: Component, fet_2: Component) -> Netlist:

     netlist = Netlist(circuit_name='FLIPPED_VOLTAGE_FOLLOWER', nodes=['VIN', 'VBULK', 'VOUT', 'Ib'])
     
     netlist.connect_netlist(fet_1.info['netlist'], [('D', 'Ib'), ('G', 'VIN'), ('S', 'VOUT'), ('B', 'VBULK')])
     netlist.connect_netlist(fet_2.info['netlist'], [('D', 'VOUT'), ('G', 'Ib'), ('S', 'VBULK'), ('B', 'VBULK')])

return netlist

### Run LVS
Design Rule Check ensures that the physical layout of an integrated circuit adheres to the manufacturing constraints defined by the foundry, such as minimum spacing, width, and enclosure rules. `Magic` is the tool we use for DRC here.

## Extraction and Post-Pex Simulation